Libs

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

Functions

In [ ]:
def calcs_operations(df, tk):

    df = df[df['ticker'] == tk] # select only the ticker (tk)
    df = df[['date', 'quantity', 'price']] # drop ticker column
    df = df.sort_values(by="date") # sort by date as ascending
    df['amount'] = df['quantity'].cumsum() # cumulative sum of quantity
    df['value'] = df['price'] * df['quantity'] # value of each operation
    df['balance'] = df['value'].cumsum() # cumulative sum of value
    df['mean_price'] = df['balance'] / df['amount'] # mean price of the investment

    return df

In [ ]:
def data_yf(tk, df):

    start = df['date'].iloc[0] # date of the first operation
    end = pd.to_datetime("today").normalize() # date of today
    ndays = (end - start).days

    data = yf.Ticker(tk)
    data = data.history(period=f"{ndays}d", start=start, end=end)
    
    return data

In [ ]:
def calcs_return(data):

    # Calculate adjustment factor
    data['Adjustment Factor'] = (1 + data['Dividends'] / data['Close']).cumprod()

    # Calculate Adjusted Close
    data['Adj Close'] = data['Close'] / data['Adjustment Factor']

    # Calculate returns
    data['Auto_pct'] = data['Adj Close'].pct_change()
    
    # Calculate cumulative returns
    data['Auto_cum'] = (1 + data['Auto_pct']).cumprod()

    return data

In [ ]:
def plot_operations(df, tk, data):

    plt.figure(figsize=(10, 5))

    # Plot the ETF value data
    data['Close'].plot(title=f'ETF - {tk}', 
                            color='blue', 
                            label='Close Price - USD')

    # Plot the range of prices
    plt.fill_between(data.index, 
                    data['High'], 
                    data['Low'], 
                    color='blue', 
                    alpha=0.25, 
                    label='High-Low Range')

    # Plot the operations
    colors = ['red' if q < 0 else 'green' for q in df['quantity']]
    plt.scatter(df['date'], 
                df['price'], 
                s=abs(df['quantity']) * 1000, 
                color=colors, 
                label='Operations (Size by Quantity)', 
                alpha=0.75)

    # Plot the mean price
    plt.axhline(y=df['mean_price'].iloc[-1], 
                color='red',
                linestyle='--', 
                label='Most Recent Mean Price')

    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def plot_return(data, tk, data_sp500, data_ibov):

    plt.figure(figsize=(10, 5))

    plt.plot(data.index, 
             data['Auto_cum']*100, 
             linestyle='-', 
             color='blue', 
             label=tk)

    plt.plot(data_sp500.index, 
             data_sp500['Auto_cum']*100, 
             linestyle='--', 
             color='red',
             label='S&P 500')

    plt.plot(data_ibov.index, 
             data_ibov['Auto_cum']*100, 
             linestyle='--', 
             color='green',
             label='IBOV')
    
    plt.title('Cumulative Daily Percentage Change in ETF vs INDEX Price')
    plt.xlabel('Date')
    plt.ylabel('Performance (%)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

Application

In [ ]:
TICKER = 'QQQM'

In [ ]:
operations_data = pd.read_excel("operations_data.xlsx")
df_calcs_operations = calcs_operations(operations_data, TICKER)
display(df_calcs_operations)

In [ ]:
df_yf = data_yf(TICKER, df_calcs_operations)
df_return = calcs_return(df_yf)
display(df_return)

In [ ]:
df_sp500_yf = calcs_return(data_yf('^GSPC', df_calcs_operations))
df_ibov_yf = calcs_return(data_yf('^BVSP', df_calcs_operations))

In [ ]:
plot_operations(df_calcs_operations, TICKER, df_yf)

In [ ]:
plot_return(df_return, TICKER, df_sp500_yf, df_ibov_yf)